
# 🧪 Regex Lab: Cleaning & Extracting Survey Data
**Dataset:** `Lab 3 Survey Responses - Form Responses.csv`  

This lab gives you hands-on practice with **regular expressions** on real survey data.
You'll build patterns to **validate**, **extract**, and **normalize** text and then run quick self-checks.



## 0) Setup
- Run the cell and load the CSV file.



In [ ]:
import pandas as pd
url = "https://docs.google.com/spreadsheets/d/1TdsMmeCbvNq2GHToHbDD67qaBXZ0LKSqj5eCkuDFNtY/export?format=csv&gid=942752529"

# Read the CSV file directly from the URL
df = pd.read_csv(url)

# Display the first few rows
df.head()


,Timestamp,What is your academic year?,what section are you in?,What prerequisites did you take for this class?,What Track or Major Are You In?,What courses are you taking?,How long does it take you to get to campus from home?,What are your thoughts on the number 67?,how many cups of coffee do you drink a week?
0,9/23/2025 12:32:13,Senior,1.0,SOCY 100,Data Science,"INST 346, INST 362, INST 354, INST 447, INST 414",25 minutes,3.0,0
1,9/23/2025 12:32:20,Junior,1.0,BSOS 233,Social Data Science,"INST 366, INST 414",10 minutes,10.0,0
2,9/23/2025 12:32:33,Senior,1.0,INST 327,NaN,alot,30 minutes,10.0,10
3,9/23/2025 12:32:40,Senior,1.0,INST 327,Social Data Science,"NEUR398H, INST462, SURV400, BSCI330, INST447, ...",45 minutes,2.0,0
4,9/23/2025 12:33:15,Senior,1.0,INST 326,Data Science,"ECON422, ANTH222, INST414, INST447",1 hour,1.0,7



## 1) Start-of-string vs Anywhere ( re.match vs re.search )
**Column**: `What courses are you taking?`

**Goal**: Create two boolean columns:

- `starts_with_code`: response starts with a course code like INST346, CMSC131

- `has_any_code`: response contains a course code anywhere in the text

[A-Z]{3,4}\s*\d{3}

In [ ]:
import re
import pandas as pd

col = "What courses are you taking?"

# clean the text to avoid NaN
s = df[col].fillna("")


In [ ]:

# TODO: write a pattern for 3–4 letters + 3 digits (e.g., INST346). Hint: use \b boundaries.
pattern = r""

# A) starts_with_code → re.match
df["starts_with_code"] = s.apply(
    lambda text: bool(re.match(pattern, text, flags=re.I))   # TODO: keep case-insensitive
)

# B) has_any_code: true if pattern appears ANYWHERE (use re.search)
df["has_any_code"] = s.apply(
    lambda text: bool(re.search(pattern, text, flags=re.I))
)

df[[col, "starts_with_code", "has_any_code"]].head(10)


,What courses are you taking?,starts_with_code,has_any_code
0,"INST 346, INST 362, INST 354, INST 447, INST 414",True,True
1,"INST 366, INST 414",True,True
2,alot,True,True
3,"NEUR398H, INST462, SURV400, BSCI330, INST447, ...",True,True
4,"ECON422, ANTH222, INST414, INST447",True,True
5,"INST447, INST414, INST377, INST346",True,True
6,"INST346,INST447, INST466, INST362, BMGT430",True,True
7,"INST447, GVPT429K, FGSM310, INST366, SURV400",True,True
8,"INST 366, INST 414, INST 447",True,True
9,"INST 462,447,490",True,True



## 2) Count All Course Codes Using re.findall
**Column**: `What prerequisites did you take for this class?`

**Goal**
- Create a new column `prereq_code_count` showing how many course codes (like **INST311**, **CMSC131**, **MATH115**) each student listed.


In [ ]:
# Pattern to match course codes: 3–4 letters + 3 digits
# TODO: use boundaries \b so you don't capture partial words

pattern = r""

# Clean column (replace NaN with empty string)
s = df[col].fillna("")

df["prereq_code_count"] = s.apply(
    lambda text: len(re.findall(pattern, text, flags=re.I))
)

# Display first few results
df[[col, "prereq_code_count"]].head(10)



,What courses are you taking?,prereq_code_count
0,"INST 346, INST 362, INST 354, INST 447, INST 414",49
1,"INST 366, INST 414",19
2,alot,6
3,"NEUR398H, INST462, SURV400, BSCI330, INST447, ...",54
4,"ECON422, ANTH222, INST414, INST447",35
5,"INST447, INST414, INST377, INST346",35
6,"INST346,INST447, INST466, INST362, BMGT430",43
7,"INST447, GVPT429K, FGSM310, INST366, SURV400",45
8,"INST 366, INST 414, INST 447",29
9,"INST 462,447,490",17


# Did you notice anything in the output?

*(Hint: look at row 9)*

Addtional Hints:
- Remember that findall() restarts after the previous match ends — the second number lacks a new department prefix, so your old pattern never sees it.

- You can use a look-ahead or repetition to capture optional shared prefixes, e.g.:
`r"\b([A-Za-z]{3,4})\s*\d{3}(?:\s*,\s*\d{3})*\b"`

…but you’ll need to test and maybe expand the pattern logic to capture each individual code.

In [ ]:
# Provide your revised code here

# Then test with this dataset
tests = [
    "INST 462,447,490",
    "CMSC131,132,216",
    "MATH 115 116"
]
for t in tests:
    print(t, "→", extract_codes(t))


NameError: name 'extract_codes' is not defined


## 3) Extract Course Codes from Free-Text
Columns: `What prerequisites did you take for this class?`, `What courses are you taking?`

**Tasks**
- Case-insensitive regex for 3–4 letters + exactly 3 digits.
- Create `prereq_codes` and `current_codes` as **lists of normalized uppercase codes**.


In [ ]:


col = "What courses are you taking?"
s = df[col].fillna("")

# 3–4 letters + 3 digits, with word boundaries, case-insensitive
pattern = r""

# Vectorized extraction
df["current_codes"] = s.str.findall(pattern, flags=re.I)

# Normalize to uppercase for consistency
df["current_codes"] = df["current_codes"].apply(lambda L: [x.upper() for x in L])

df[[col, "current_codes"]].head(10)


NameError: name 'df' is not defined


## 4) Validate Academic Year (strict fullmatch)
**Column**: `year_valid`

**Goal**
- Create a boolean column year_valid that is True only when the entry is exactly one of:
`Freshman`, `Sophomore`, `Junior`, `Senior`(case-insensitive, allow surrounding spaces).

**Hints**

- Make sure inputs like " Sophomore " pass (strip spaces).

- Ensure "Senior year" or "JR" do not pass.

- If you prefer, compile once: pat = re.compile(pattern, flags=re.I) and use pat.fullmatch(...).

In [ ]:

col = "What is your academic year?"
assert col in df.columns, f"Missing column: {col}"
s = df[col].astype(str)
 #        ^(freshman|sophomore|junior|senior)$

# 1) Write a pattern that matches ONLY the 4 allowed values (case-insensitive).
#    Tips:
#    - Use ^ ... $ anchors (or re.fullmatch)
#    - Allow case-insensitive matching with (?i) or flags=re.I
#    - Consider trimming surrounding spaces with .strip()

#TODO: e.g., r"(?i)^(freshman|sophomore|junior|senior)$"
pattern = r""

# 2) Create boolean 'year_valid' using re.fullmatch in a lambda.
df["year_valid"] = s.apply(
    lambda text: bool(re.fullmatch(pattern, text.strip()))  # TODO: keep strict fullmatch
)

df[[col, "year_valid"]].head(10)



## 5) Standardize Course List Delimiters (sub → split)
**Column**: `how many cups of coffee do you drink a week?`

**Goal**
- Create `courses_tokens` by first normalizing separators (commas, semicolons, the word “and”, and `&`) to a single comma, then **splitting** into a clean list of tokens.


In [ ]:
#\s*(?:,|;|&|and)\s*

col = "What courses are you taking?"
assert col in df.columns, f"Missing column: {col}"

s = df[col].fillna("").astype(str)

# 1) Write a regex that matches the separators we want to normalize:
#    commas, semicolons, the word "and", and "&", with optional spaces around them.
#    Example targets: ",", " ; ", "and", "&"

sep_pat = r""  # TODO: e.g., r"\s*(?:,|;|&|and)\s*"

# 2) Normalize all separators to a single ", "
normalized = s.str.replace(sep_pat, ", ", regex=True).str.strip()

# 3) Remove stray leading/trailing commas created by normalization
normalized = normalized.str.strip(", ").str.strip()

# 4) Split on commas (handle extra spaces around commas)
df["courses_tokens"] = normalized.str.split(r"\s*,\s*")

df[[col, "courses_tokens"]].head(10)




## 6) Coffee: Clean + Extract Min/Max
**Column**: `how many cups of coffee do you drink a week?`
**Goal**:
- Clean free-text answers (remove filler like “about”, “~”, “cups”, “per week”).

- Extract an integer or integer range (e.g., 3-5).

- Produce two numeric columns: coffee_min, coffee_max.
  * If a single number, set both min & max to that number.
  * If no number found, set both to `None`.
  * Treat `none`/`zero` like `0`.


In [ ]:
col = "how many cups of coffee do you drink a week?"
assert col in df.columns, f"Missing column: {col}"

s = df[col].fillna("").astype(str).str.lower()

# 1) Clean filler words and symbols (keep just numbers / ranges)
#    Examples to remove: "about", "~", "cups", "cup", "per week",
clean = (
    s.str.replace(r"", "", regex=True)
     .str.replace(r"", "", regex=True)
     .str.replace(r"", "", regex=True)
     .str.replace(r"", "", regex=True)
     .str.strip()
)

# 2) Pattern for integer or integer range like "3" or "3-5"
rng_pat = r""  # TODO: e.g., r"\b\d+(?:\s*-\s*\d+)?\b"

# 3) Extract the first number/range if present
m = clean.str.extract(rng_pat)

# 4) Convert to min/max:
#    - If it's a range like "3-5", split and convert
#    - If it's a single number, set both to that number
#Provide your code here:
def to_min_max(token):


# 5) Handle words like "none" or "zero" when no digits are found
df.loc[(df["coffee_min"].isna()) & s.str.contains(r"\b(none|zero)\b", na=False), ["coffee_min","coffee_max"]] = (0, 0)

df[[col, "coffee_min", "coffee_max"]].head(10)



## 7) Split Prerequisites into Tokens (re.split)

**Column**:`What prerequisites did you take for this class?`

**Goal**:Tokenize the text into a clean list by splitting on commas, semicolons, and the word **“and”** (also handle `&`).

**Optionally**, strip and drop empty tokens.


In [ ]:
col = "What prerequisites did you take for this class?"
assert col in df.columns, f"Missing column: {col}"

s = df[col].fillna("").astype(str)

# 1) Separator pattern (commas, semicolons, 'and', '&') with optional spaces
sep_pat = r""  # TODO: e.g., r"\s*(?:,|;|&|and)\s*"

# 2) Normalize separators to a single comma
normalized = s.str.replace(sep_pat, ", ", regex=True, case=False).str.strip()

# 3) Remove leading/trailing commas that may result from normalization
normalized = normalized.str.strip(", ").str.strip()

# 4) Split on commas into tokens (handle spaces)
df["prereq_tokens"] = normalized.str.split(r"\s*,\s*")

# 5) OPTIONAL: drop empty tokens
# df["prereq_tokens"] = df["prereq_tokens"].apply(lambda xs: [t for t in xs if t])

df[[col, "prereq_tokens"]].head(10)




## 8) Advanced: Lookarounds & Backreferences
1) Positive lookahead: rows in `What courses are you taking?` that mention `INST` **and** a 300-level course.  
2) Backreference: in `What prerequisites...`, same department code appears twice.


**Task A**: Poaitive Lookahead
**Goal**:
Find rows in `What courses are you taking?` that mention:
- The department INST, and
- At least one 300-level course (e.g., INST346, CMSC320, etc.)

**Hints**

* `(?=...)` is a **positive lookahead** — it checks that the pattern exists ahead but doesn’t consume text.  

* Combine multiple lookaheads for “and” conditions.  

* `\w{3,4}3\d{2}` matches codes like `INST346`, `CMSC320`.

In [ ]:
#Provide your code here:


**Task B**:  Backreference
**Goal**:
In `What prerequisites did you take for this class?` find rows where the same department (e.g., INST) appears twice with different course numbers (e.g., INST311 ... INST346).

**Hints**

* `\1` refers to the **first** captured group.  
* Case-insensitive flag (`(?i)`) ensures `inst` = `INST`.  
* `.*` allows any text (even across spaces or punctuation) between the two matches.  

In [ ]:
#Provide your code here:
